Settings

In [1]:
import zarr
import numpy as np
from dataset_util import get_value_target
from training.train_cnn import fill_default_config
from preprocessing.game_phase_detector import *


read_path = "/PommerLearn/1K_mixed_0.zr"
write_path = "/PommerLearn/1K_mixed_0"


/opt/miniconda3/envs/pommer/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import error GL! You will not be able to render --> Library "GL" not found.


In [4]:
z = zarr.open(read_path)

chunks = z['act'].chunks

KeyError: 'act'

Generate Values

In [ ]:
train_config = fill_default_config({
    "device":6,
})

val_target = get_value_target(
    z,
    discount_factor=train_config["discount_factor"], 
    mcts_val_weight=train_config["mcts_val_weight"]
    )
assert val_target.shape == z['val'].shape, "Shapes do not line up"

z['val']= val_target


Add Phase Values

In [3]:


get_phase_vector(z)

KeyError: 'obs'

Split into three seperate zarrs

In [ ]:
for i in range(4):
    act = z['act'][:]
    obs = z['obs'][:]
    phase = z['phase'][:]
    pol = z['pol'][:]
    q = z['q'][:]
    val = z['val'][:]

    mask = phase == i

    act = act[mask]
    obs = obs[mask]
    phase = phase[mask]
    pol = pol[mask]
    q = q[mask]
    val = val[mask]

    new_zarr = zarr.open(write_path+'_phase_'+str(i)+'.zr', mode='w')
    new_zarr.create_dataset("act", data=act, chunks=chunks)
    new_zarr.create_dataset("obs", data=obs, chunks=chunks)
    new_zarr.create_dataset("phase", data=phase, chunks=chunks)
    new_zarr.create_dataset("pol", data=pol, chunks=chunks)
    new_zarr.create_dataset("q", data=q, chunks=chunks)
    new_zarr.create_dataset("val", data=val, chunks=chunks)

    new_zarr.attrs['Steps'] = act.shape[0]
    new_zarr.attrs['AgentSteps'] = z.attrs['AgentSteps']
    new_zarr.attrs['EpisodeSteps'] = z.attrs['EpisodeSteps']
    new_zarr.attrs['AgentIds'] = z.attrs['AgentIds']
    new_zarr.attrs['AgentEpisode'] = z.attrs['AgentEpisode']
    new_zarr.attrs['EpisodeDead'] = z.attrs['EpisodeDead']
    new_zarr.attrs['EpisodeWinner'] = z.attrs['EpisodeWinner']
    new_zarr.attrs['EpisodeGameMode'] = z.attrs['EpisodeGameMode']
    new_zarr.attrs['EpisodeDraw'] = z.attrs['EpisodeDraw']
    new_zarr.attrs['EpisodeWinningTeam'] = z.attrs['EpisodeWinningTeam']
    new_zarr.attrs['EpisodeDone'] = z.attrs['EpisodeDone']
    new_zarr.attrs['EpisodeActions'] = z.attrs['EpisodeActions']


    print("Phase {} has {} items".format(i, act.shape[0])  )
